In [21]:
# Load Dependencies

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import time
import json
import csv
import os

In [22]:
# Setup Selenium with Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [23]:
# Visit the USGS search results page
base_url = "https://astrogeology.usgs.gov"
search_url = f"{base_url}/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver.get(search_url)
time.sleep(2)

In [24]:
# Parse with BeautifulSoup
soup = bs(driver.page_source, 'html.parser')
items = soup.find_all('div', class_='description')

In [25]:
# Save to Dictionary
hemisphere_image_urls = []

In [26]:
for item in items:
    # Extract and clean the title
    raw_title = item.find('h3').text.strip()
    title = raw_title.replace(" Enhanced", "").replace("Hemisphere", "Hemisphere").strip()

    # Build full subpage URL
    partial_url = item.find('a')['href']
    full_link = base_url + partial_url

    # Navigate to subpage
    driver.get(full_link)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'downloads')))

    # Parse subpage
    sub_soup = bs(driver.page_source, 'html.parser')
    img_url = sub_soup.find('a', text='Sample')['href']

    # Append to list
    hemisphere_image_urls.append({
        'title': title,
        'page_url': full_link,
        'img_url': img_url
    })


In [27]:
import json
print(json.dumps(hemisphere_image_urls, indent=2))


[]


In [28]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time

hemisphere_image_urls = []
base_url = "https://astrogeology.usgs.gov"

# Let the page load and parse
soup = bs(driver.page_source, 'html.parser')
items = soup.find_all('div', class_='item')

# Sanity check
print(f"Found {len(items)} items")

for item in items:
    try:
        # Build the full link to the hemisphere page
        partial_url = item.find('a')['href']
        full_link = base_url + partial_url

        # Go to the hemisphere's detail page
        driver.get(full_link)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
        time.sleep(1)  # Slight delay to ensure page is loaded

        # Parse the subpage
        sub_soup = bs(driver.page_source, 'html.parser')
        title = sub_soup.find('h2', class_='title').text.replace(' Enhanced', '').strip()
        img_url = sub_soup.find('a', text='Sample')['href']

        # Store in list
        hemisphere_image_urls.append({
            'title': title,
            'img_url': img_url
        })
        print(f"Added: {title}")

    except Exception as e:
        print(f"Error processing item: {e}")

# Final check
print(hemisphere_image_urls)


Found 0 items
[]


# Debugging For Loop

In [29]:
print(driver.current_url)
print(driver.title)


https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars



In [30]:
from bs4 import BeautifulSoup as bs

soup = bs(driver.page_source, 'html.parser')
items = soup.find_all('div', class_='item')
print(f"Items found: {len(items)}")

# Optional: show one item’s raw HTML
if items:
    print(items[0].prettify())


Items found: 0


In [31]:
for item in items:
    try:
        a_tag = item.find('a')
        href = a_tag['href']
        print(f"Found href: {href}")
    except Exception as e:
        print(f"Error extracting href: {e}")


In [32]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Wait for the 'item' divs to load on the page
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'item')))


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001055b82c8 chromedriver + 6197960
1   chromedriver                        0x00000001055af8ea chromedriver + 6162666
2   chromedriver                        0x0000000105034de0 chromedriver + 417248
3   chromedriver                        0x0000000105086797 chromedriver + 751511
4   chromedriver                        0x00000001050869b1 chromedriver + 752049
5   chromedriver                        0x00000001050d69b4 chromedriver + 1079732
6   chromedriver                        0x00000001050ac9ed chromedriver + 907757
7   chromedriver                        0x00000001050d3cdb chromedriver + 1068251
8   chromedriver                        0x00000001050ac793 chromedriver + 907155
9   chromedriver                        0x0000000105078b25 chromedriver + 695077
10  chromedriver                        0x0000000105079791 chromedriver + 698257
11  chromedriver                        0x0000000105574cc0 chrome

In [33]:
print("Current page URL:", driver.current_url)
print("Page title:", driver.title)

# Wait a few seconds manually, then fetch the full source
import time
time.sleep(5)
html = driver.page_source
print(html[:1000])  # Print first 1000 chars for inspection


Current page URL: https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
Page title: 
<html><head><meta name="color-scheme" content="light dark"><meta charset="utf-8"></head><body><pre>{"collections":[{"abstract":"This page introduces the Kaguya Multiband Imager derived spectral and derived mineral maps by the Japan Aerospace Exploration Agency (JAXA) and the University of Hawaii. The mosaics were created from topographically-corrected MI","geoform":["Collection"],"name":"lunar-kaguya-multiband-imager-mosaics","onlink":"https://astrogeology.usgs.gov/search/map/lunar-kaguya-multiband-imager-mosaics","pubdate":"2015-01-01","thumb":"https://astrogeology.usgs.gov/ckan/dataset/a53c9dbf-e03a-4d7a-8641-fc7af97a75c2/resource/a7e1a610-f8ef-4080-99e0-1b637d6c269f/download/moon-selene-kaguya-mi-thumb.jpg","title":"Lunar Kaguya Multiband Imager Mosaics"},{"abstract":"&lt;span class=\"intro\"&gt;The following products were the first step of cartography planning&lt;/spa

In [34]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'h3')))


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001055b82c8 chromedriver + 6197960
1   chromedriver                        0x00000001055af8ea chromedriver + 6162666
2   chromedriver                        0x0000000105034de0 chromedriver + 417248
3   chromedriver                        0x0000000105086797 chromedriver + 751511
4   chromedriver                        0x00000001050869b1 chromedriver + 752049
5   chromedriver                        0x00000001050d69b4 chromedriver + 1079732
6   chromedriver                        0x00000001050ac9ed chromedriver + 907757
7   chromedriver                        0x00000001050d3cdb chromedriver + 1068251
8   chromedriver                        0x00000001050ac793 chromedriver + 907155
9   chromedriver                        0x0000000105078b25 chromedriver + 695077
10  chromedriver                        0x0000000105079791 chromedriver + 698257
11  chromedriver                        0x0000000105574cc0 chrome